# Loading the data

In [1]:
import numpy as np

In [2]:
dataset = np.load("./1.npy")

In [3]:
dataset.shape

(259520, 65)

# Convert your dataset into `segments`

In [4]:
def extract(input, n_classes, n_fea, time_window, moving):
    xx = input[:, :n_fea]
    yy = input[:, n_fea:n_fea + 1]
    new_x = []
    new_y = []
    number = int((xx.shape[0] / moving) - 1)
    for i in range(number):
        ave_y = np.average(yy[int(i * moving):int(i * moving + time_window)])
        if ave_y in range(n_classes + 1):
            new_x.append(xx[int(i * moving):int(i * moving + time_window), :])
            new_y.append(ave_y)

    new_x = np.array(new_x)
    new_x = new_x.reshape([-1, n_fea * time_window])
    new_y = np.array(new_y)
    new_y.shape = [new_y.shape[0], 1]
    data = np.hstack((new_x, new_y))
    data = np.vstack((data, data[-1]))  # add the last sample again, to make the sample number round
    return data

In [5]:
data = extract(dataset ,11 ,64 ,16 ,8)

# How to Divide Data into Training and Testing Sets

In [6]:
from sklearn.model_selection import train_test_split
n_fea = 64
time_window = 16
no_longfeature = n_fea * time_window
data_seg_feature = data[:, :no_longfeature]
data_seg_label = data[:, no_longfeature:no_longfeature+1]
train_feature, test_feature, train_label, test_label = train_test_split(data_seg_feature, data_seg_label,test_size=0.2, shuffle=True)

In [7]:
train_feature.shape

(25807, 1024)

In [8]:
test_feature.shape

(6452, 1024)

# Data Preprocessing

In [9]:
from sklearn.preprocessing import StandardScaler
scaler1 = StandardScaler().fit(train_feature)
train_fea_norm1 = scaler1.transform(train_feature) # normalize the training data
test_fea_norm1 = scaler1.transform(test_feature) # normalize the test data

# Building a CNN Network

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [11]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels=1,
                out_channels=16,
                kernel_size=(2,4),
                stride=1,
                padding= (1,2)  #([1,2]-1)/2,
            ),
            nn.ReLU(),
            nn.MaxPool2d((2,4))
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, (2,2), stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d((2, 2))
        )
        self.fc = nn.Linear(1024, 128)  # 64*2*4
        self.out = nn.Linear(128, n_class)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)

        x = F.relu(self.fc(x))
        output = self.out(x)
        return x